## 安装依赖

In [1]:
%pip install -q matplotlib opencv-python torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille ipywidgets openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 重新加载autoreload扩展，使代码修改后自动生效
%reload_ext autoreload
# 设置autoreload模式为2，自动重新加载所有导入的模块
%autoreload 2

In [3]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

c:\Users\k\.conda\envs\rag\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\k\.cache\huggingface\hub\datasets--m-ric--huggingface_doc. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/2647 [00:00<?, ?it/s]

In [24]:
# from huggingface_hub import InferenceClient


# repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# llm_client = InferenceClient(
#     model=repo_id,
#     timeout=120,
# )


# def call_llm(inference_client: InferenceClient, prompt: str):
#     response = inference_client.post(
#         json={
#             "inputs": prompt,
#             "parameters": {"max_new_tokens": 1000},
#             "task": "text-generation",
#         },
#     )
#     return json.loads(response.decode())[0]["generated_text"]


# call_llm(llm_client, "This is a test context")
import openai
import os
import dotenv
# 建议将 API 密钥存储在环境变量中
from dotenv import load_dotenv
load_dotenv()

model_name = "THUDM/GLM-4-32B-0414"

# 或者直接设置（不推荐用于生产环境）
# openai.api_key = "YOUR_OPENAI_API_KEY"

# 初始化 OpenAI 客户端
# 如果您使用的是 Azure OpenAI，请按如下方式初始化：
llm_client = openai.OpenAI(
    api_key=os.getenv("API_KEY"),
    base_url = os.getenv("BASE_URL"),
)
# 否则，使用默认的 OpenAI 初始化：
# client = openai.OpenAI()



def call_llm(openai_client: openai.OpenAI, prompt: str, model: str = "THUDM/GLM-4-32B-0414"):
    """
    使用指定的 OpenAI 模型生成文本。

    Args:
        openai_client: 初始化后的 OpenAI 客户端实例。
        prompt: 输入给模型的提示文本。
        model: 要使用的模型名称，比如：THUDM/GLM-4-32B-0414.

    Returns:
        模型生成的文本内容。
    """
    try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000, # 根据需要调整
            temperature=0.7 # 根据需要调整
        )
        # 检查 response.choices 是否为空以及第一个 choice 是否存在
        if response.choices and len(response.choices) > 0 and response.choices[0].message:
             return response.choices[0].message.content.strip()
        else:
             print("Warning: OpenAI API returned an empty response or invalid structure.")
             return "" # 或者可以抛出异常
    except Exception as e:
        print(f"An error occurred while calling the OpenAI API: {e}")
        return "" # 或者根据错误处理策略返回 None 或抛出异常


test_prompt = "解释一下什么是 RAG？"
generated_text = call_llm(llm_client, test_prompt)
print(generated_text)



RAG 是 Retrieval-Augmented Generation 的缩写，中文翻译为“检索增强生成”。这是一个结合了信息检索和文本生成技术的自然语言处理（NLP）模型架构。RAG 模型通过检索外部知识库（如文档库、知识图谱等）来增强其生成文本的能力，从而生成更准确、更连贯、更符合上下文的文本。

RAG 模型通常包含两个主要组件：

1. 检索器（Retriever）：负责从外部知识库中检索与输入查询相关的文档或片段。检索器通常使用信息检索技术，如向量检索、关键词匹配等。

2. 生成器（Generator）：负责根据检索到的文档和输入查询生成文本。生成器通常是基于大型预训练语言模型的，如GPT、BERT等。

RAG 模型的工作流程如下：

1. 输入查询：用户输入一个查询或问题。

2. 检索相关文档：检索器根据输入查询从外部知识库中检索相关的文档或片段。

3. 生成文本：生成器根据检索到的文档和输入查询生成文本。

4. 输出结果：将生成的文本作为结果返回给用户。

RAG 模型的优势在于，它能够利用外部知识库的信息来增强生成文本的能力，从而在许多任务中取得更好的性能。例如，在问答系统中，RAG 模型可以检索相关文档来回答用户的问题，从而提高答案的准确性和完整性。在文本生成任务中，RAG 模型可以检索相关文档来提供背景信息，从而生成更连贯、更符合上下文的文本。


In [ ]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

ZH_QA_generation_prompt = """

以下是中文版指示：
你的任务是根据给定的上下文编写一个事实性问题及其答案。
你的事实性问题应当能够通过上下文中的具体、简洁的事实信息来回答。
你的事实性问题应当以用户在搜索引擎中提问的风格来表述。
这意味着你的事实性问题不能包含"根据文章"或"上下文"等表述。

请按照以下格式提供你的回答：

Output:::
Factoid question: (你的事实性问题)
Answer: (你对该事实性问题的回答)

以下是上下文：

Context: {context}\n
Output:::"""

In [ ]:
import random

N_GENERATIONS = 10  # 我们在这里有意只生成10个QA对，考虑到成本和时间因素

print(f"生成 {N_GENERATIONS} 个QA对...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # 生成QA对
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=sampled_context.page_content))
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "答案太长"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 10 QA couples...


  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
display(pd.DataFrame(outputs).head(1))


,context,question,answer,source_doc
0,"<Tip>\n\nGuess mode does not have any impact on prompt conditioning and you can still provide a prompt if you want.\n\n</Tip>\n\nSet `guess_mode=True` in the pipeline, and it is [recommended](https://github.com/lllyasviel/ControlNet#guess-mode--non-prompt-mode) to set the `guidance_scale` value between 3.0 and 5.0.\n\n```py\nfrom diffusers import StableDiffusionControlNetPipeline, ControlNetModel\nfrom diffusers.utils import load_image, make_image_grid\nimport numpy as np\nimport torch\nfrom PIL import Image\nimport cv2\n\ncontrolnet = ControlNetModel.from_pretrained(""lllyasviel/sd-controlnet-canny"", use_safetensors=True)\npipe = StableDiffusionControlNetPipeline.from_pretrained(""runwayml/stable-diffusion-v1-5"", controlnet=controlnet, use_safetensors=True).to(""cuda"")\n\noriginal_image = load_image(""https://huggingface.co/takuma104/controlnet_dev/resolve/main/bird_512x512.png"")\n\nimage = np.array(original_image)\n\nlow_threshold = 100\nhigh_threshold = 200\n\nimage = cv2.Canny(image, low_threshold, high_threshold)\nimage = image[:, :, None]\nimage = np.concatenate([image, image, image], axis=2)\ncanny_image = Image.fromarray(image)\n\nimage = pipe("""", image=canny_image, guess_mode=True, guidance_scale=3.0).images[0]\nmake_image_grid([original_image, canny_image, image], rows=1, cols=3)\n```\n\n<div class=""flex gap-4"">\n <div>\n <img class=""rounded-xl"" src=""https://huggingface.co/takuma104/controlnet_dev/resolve/main/gen_compare_guess_mode/output_images/diffusers/output_bird_canny_0.png""/>\n <figcaption class=""mt-2 text-center text-sm text-gray-500"">regular mode with prompt</figcaption>\n </div>\n <div>\n <img class=""rounded-xl"" src=""https://huggingface.co/takuma104/controlnet_dev/resolve/main/gen_compare_guess_mode/output_images/diffusers/output_bird_canny_0_gm.png""/>\n <figcaption class=""mt-2 text-center text-sm text-gray-500"">guess mode without prompt</figcaption>\n </div>\n</div>\n\n## ControlNet with Stable Diffusion XL",What is the recommended guidance scale value for guess mode in StableDiffusionControlNetPipeline?\n,Between 3.0 and 5.0,huggingface/diffusers/blob/main/docs/source/en/using-diffusers/controlnet.md


## 设置批判智能体
1. 具体性（Groundedness）：问题是否可以从给定的上下文中得到回答？
2. 相关性（Relevance）：问题对用户是否相关？例如，"transformers 4.29.1 发布的日期是什么？"对于 ML 用户来说并不相关。
3. 独立（Stand-alone）：对于一个具有领域知识/互联网访问权限的人来说，问题在没有任何上下文的情况下是否可以理解？与此相反的是，对于从特定博客文章生成的问题比如”这篇文章中使用的函数是什么？”

In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independant from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

# 作用：让 LLM 判断提供的问题是否能仅根据给定的上下文明确回答。
# 评估维度：“扎根性”（Groundedness）。
# 评分标准：1（根据上下文完全无法回答）到 5（根据上下文可以清晰明确地回答）。

zh_question_groundedness_critique_prompt = """
你将得到一个上下文和一个问题。
你的任务是提供一个“总评分”，评估在给定上下文的情况下，问题能在多大程度上得到明确的回答。
请按1到5的等级评分，其中1表示根据上下文完全无法回答该问题，5表示根据上下文可以清晰明确地回答该问题。

请按以下格式提供你的回答：

回答:::
评价理由: (你给出评分的理由，文本形式)
总评分: (你的评分，1到5之间的数字)

你必须在回答中提供“评价理由:”和“总评分:”的值。

以下是问题和上下文。

问题: {question}\n
上下文: {context}\n
回答::: """



# 作用：让 LLM 判断提供的问题对于使用 Hugging Face 生态系统构建 NLP 应用的机器学习开发者来说是否有用。
# 评估维度：“相关性”（Relevance）。
# 评分标准：1（该问题完全没有用处）到 5（该问题非常有用）。

zh_question_relevance_critique_prompt = """
你将得到一个问题。
你的任务是提供一个“总评分”，代表这个问题对于使用Hugging Face生态系统构建NLP应用的机器学习开发者有多大用处。
请按1到5的等级评分，其中1表示该问题完全没有用处，5表示该问题非常有用。

请按以下格式提供你的回答：

回答:::
评价理由: (你给出评分的理由，文本形式)
总评分: (你的评分，1到5之间的数字)

你必须在回答中提供“评价理由:”和“总评分:”的值。

以下是问题。

问题: {question}\n
回答::: """

# 作用：让 LLM 判断提供的问题是否能在脱离原始上下文的情况下被理解。
# 评估维度：“独立性”（Stand-alone）。
# 评分标准：1（该问题需要依赖额外信息才能被理解）到 5（该问题本身就有意义）。特别注意，即使问题包含专业术语，只要能通过查阅文档理解，也可评为 5 分；但如果问题中隐含提及“上下文”或“文档中”，则必须评为 1 分。

zh_question_standalone_critique_prompt = """
你将得到一个问题。
你的任务是提供一个“总评分”，代表这个问题在多大程度上是独立于上下文的。
请按1到5的等级评分，其中1表示该问题需要依赖额外信息才能被理解，5表示该问题本身就有意义。
例如，如果问题提到了特定的环境，如“在上下文中”或“在文档中”，评分必须为1。
问题可以包含像Gradio、Hub、Hugging Face或Space这样晦涩的技术名词或缩写，并且仍然可以评为5分：只要对于一个能够查阅文档的操作员来说，问题的含义是清晰的即可。

例如，“ViT模型是从哪个检查点导入的？”应评为1分，因为它隐含地提到了一个上下文，因此问题并非独立于上下文。

请按以下格式提供你的回答：

回答:::
评价理由: (你给出评分的理由，文本形式)
总评分: (你的评分，1到5之间的数字)

你必须在回答中提供“评价理由:”和“总评分:”的值。

以下是问题。

问题: {question}\n
回答::: """


In [17]:
print("正在为每个QA对生成评价...")
for output in tqdm(outputs):
    # 使用中文版的评价提示
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            zh_question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
        ),
        "relevance": call_llm(
            llm_client,
            zh_question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            zh_question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            # 从评价结果中提取评分和评价理由
            # 对于中文评价，我们需要查找"总评分:"和"评价理由:"
            if "总评分:" in evaluation:
                score = int(evaluation.split("总评分:")[-1].strip())
                eval = evaluation.split("总评分:")[0].split("评价理由:")[-1].strip()
            else:
                # 如果找不到中文标识，回退到英文格式
                score = int(evaluation.split("Total rating:")[-1].strip())
                eval = evaluation.split("Total rating:")[-2].split("Evaluation:")[-1].strip()
                
            # 更新输出字典，添加评分和评价
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        print(f"处理评价时出现错误: {e}")
        continue

正在为每个QA对生成评价...


  0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

c:\Users\k\.conda\envs\rag\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\k\.cache\huggingface\hub\datasets--m-ric--huggingface_doc_qa_eval. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65 [00:00<?, ? examples/s]

In [19]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)
]

  0%|          | 0/2647 [00:00<?, ?it/s]

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    将文档分割成大小为 `chunk_size` 个字符的块，并返回文档列表。
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10), # 设置块重叠大小为块大小的10%
        add_start_index=True, # 添加块在原始文档中的起始索引
        strip_whitespace=True, # 去除块首尾的空白字符
        separators=["\n\n", "\n", ".", " ", ""], # 用于分割文本的分隔符列表
    )

    docs_processed = []
    for doc in knowledge_base: # 遍历知识库中的每个文档
        docs_processed += text_splitter.split_documents([doc]) # 将当前文档分割成块并添加到处理后的文档列表中

    # 移除重复项
    unique_texts = {} # 用于存储已处理过的唯一文本内容
    docs_processed_unique = [] # 用于存储去重后的文档块
    for doc in docs_processed: # 遍历所有处理后的文档块
        if doc.page_content not in unique_texts: # 如果当前块的内容不在唯一文本字典中
            unique_texts[doc.page_content] = True # 将当前块的内容添加到唯一文本字典中
            docs_processed_unique.append(doc) # 将当前块添加到去重后的文档块列表中

    return docs_processed_unique # 返回去重后的文档块列表

In [22]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = r"C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": True},  # set True to compute cosine similarity
    )

    # Check if embeddings already exist on disk
    index_name = f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

In [ ]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

zh_RAG_PROMPT_TEMPLATE = """
<|system|>
请根据上下文信息，对问题给出全面回答。
仅回答被问到的问题，回答应简洁并与问题相关。
在相关时，请提供源文档的编号。
如果无法从上下文中推断出答案，请不要给出答案。</s>
<|user|>
上下文:
{context}
---
现在，这是你需要回答的问题。

问题: {question}
</s>
<|assistant|>
"""

In [32]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


def answer_with_rag(
    question: str,
    llm_client: openai.OpenAI,
    knowledge_index_path: str = None,
    embedding_model: str = r"C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3",
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
    use_chinese: bool = False,
    model_name: str = "THUDM/GLM-4-32B-0414"
) -> Tuple[str, List[dict]]:
    """
    使用RAG方法回答问题，基于FAISS索引和Sentence Transformers
    
    Args:
        question: 用户提出的问题
        llm_client: OpenAI客户端实例
        knowledge_index_path: FAISS索引的路径，如果为None则需要重新构建
        embedding_model: 用于文本嵌入的sentence-transformers模型
        num_retrieved_docs: 初始检索的文档数量
        num_docs_final: 最终用于生成答案的文档数量
        use_chinese: 是否使用中文提示模板
        model_name: 大模型的名称
        
    Returns:
        生成的答案文本和相关的检索文档列表
    """
    # 初始化或加载文本嵌入模型
    embed_model = SentenceTransformer(embedding_model)
    
    # 如果没有提供知识库索引路径，从RAW_KNOWLEDGE_BASE构建嵌入并创建索引
    if knowledge_index_path is None:
        # 从RAW_KNOWLEDGE_BASE中提取文档并进行分割
        # 这里假设RAW_KNOWLEDGE_BASE已经被定义并包含了所有需要的文档
        doc_texts = [doc.page_content for doc in docs_processed]
        doc_embeddings = embed_model.encode(doc_texts, show_progress_bar=True, convert_to_tensor=True)
        
        # 创建FAISS索引
        dimension = doc_embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(doc_embeddings.cpu().numpy())
    else:
        # 从文件加载FAISS索引
        index = faiss.read_index(knowledge_index_path)
        doc_texts = [doc.page_content for doc in docs_processed]  # 假设docs_processed已经定义
    
    # 生成问题的嵌入
    question_embedding = embed_model.encode(question, convert_to_tensor=True)
    
    # 检索相关文档
    distances, indices = index.search(question_embedding.cpu().numpy().reshape(1, -1), k=num_retrieved_docs)
    retrieved_docs = [doc_texts[i] for i in indices[0]]
    
    # 限制最终使用的文档数量
    relevant_docs = retrieved_docs[:num_docs_final]
    
    # 构建上下文
    context = "\n检索到的文档:\n" if use_chinese else "\nExtracted documents:\n"
    context += "".join([f"文档 {str(i)}:::\n" + doc + "\n\n" if use_chinese 
                        else f"Document {str(i)}:::\n" + doc + "\n\n" 
                        for i, doc in enumerate(relevant_docs)])

    # 根据语言选择提示模板
    prompt_template = zh_RAG_PROMPT_TEMPLATE if use_chinese else RAG_PROMPT_TEMPLATE
    final_prompt = prompt_template.format(question=question, context=context)

    # 生成答案
    response = llm_client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": final_prompt}
        ],
        max_tokens=1000,
        temperature=0.1
    )
    answer = response.choices[0].message.content.strip()
    
    # 返回答案和使用的文档
    retrieved_doc_objects = [{"page_content": doc} for doc in relevant_docs]
    return answer, retrieved_doc_objects

In [33]:
import os
import json
from tqdm.auto import tqdm


def run_rag_tests(
    eval_dataset,
    llm_client,
    output_file="rag_test_results.json",
    embedding_model=r"C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3",
    knowledge_index_path=None,
    num_retrieved_docs=30,
    num_docs_final=7,
    verbose=True,
    test_settings=None,
    use_chinese=False,
    model_name="THUDM/GLM-4-32B-0414"
):
    """
    在给定数据集上运行RAG测试，并将结果保存到指定的输出文件。
    
    Args:
        eval_dataset: 评估数据集
        llm_client: 用于生成回答的OpenAI客户端
        output_file: 保存结果的文件路径
        embedding_model: 用于文本嵌入的模型路径
        knowledge_index_path: FAISS索引的路径，如果为None则需要重新构建
        num_retrieved_docs: 初始检索的文档数量
        num_docs_final: 最终用于生成答案的文档数量
        verbose: 是否打印详细输出
        test_settings: 测试设置的描述信息
        use_chinese: 是否使用中文提示模板
        model_name: 大模型的名称
    """
    try:  # 如果输出文件已存在，加载之前的生成结果
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        # 使用RAG回答问题
        answer, relevant_docs = answer_with_rag(
            question,
            llm_client,
            knowledge_index_path,
            embedding_model=embedding_model,
            num_retrieved_docs=num_retrieved_docs,
            num_docs_final=num_docs_final,
            use_chinese=use_chinese,
            model_name=model_name
        )
        
        if verbose:
            print("=======================================================")
            print(f"问题: {question}" if use_chinese else f"Question: {question}")
            print(f"回答: {answer}" if use_chinese else f"Answer: {answer}")
            print(f"正确答案: {example['answer']}" if use_chinese else f"True answer: {example['answer']}")
            
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc["page_content"] for doc in relevant_docs],  # 只保存文本内容
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        # 实时保存结果，以防程序中断
        with open(output_file, "w") as f:
            json.dump(outputs, f, ensure_ascii=False, indent=2)  # 使用ensure_ascii=False以正确保存中文